# Question 5.1

In [ ]:
import plotly.graph_objects as go

seasons = df['Season'].drop_duplicates().to_list()

def get_plot_data(df, season_to_plot):
    filtered_df = df[df['Season'] == season_to_plot]
    # Group by the type of the shot and the Event (goal or shot(miss)) and count the occurrences of each combination
    data_to_plot = filtered_df.groupby(['ShotType', 'Event']).size().unstack(fill_value=0)
    # rename the columns
    data_to_plot.columns = ['Goals', 'Shots']
    # Calculate the total number of shots + goals for each shot type
    data_to_plot['Total'] = data_to_plot['Shots'] + data_to_plot['Goals']
    return data_to_plot
# get_plot_data(df, '2016-17')
def get_plot_button(season_id, df, seasons):
    button = dict(
        args=[{ "y": [get_plot_data(df, seasons[season_id])['Goals'], get_plot_data(df, seasons[season_id])['Total']]  }],
        label=seasons[season_id],
        method="update"
        )
    return button

index = get_plot_data(df, seasons[0]).index

fig = go.Figure()

fig.add_trace(
    go.Bar(x=index, y=get_plot_data(df, seasons[0])['Goals'], name="Goals", marker_color="#008B02")
    )
fig.add_trace(
    go.Bar(x=index, y=get_plot_data(df, seasons[0])['Total'], name="Total", marker_color="#B80000")
    )

fig.update_layout(
    title={
        'text': "Comparaison of the shot type",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Shot Type",
    yaxis_title="Number of shots",
    barmode='stack',
    width=1000,
    height=500,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)

fig.update_layout(
    updatemenus=[
        dict(
            buttons=[get_plot_button(i, df, seasons) for i in range(len(seasons))],
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
            ),
        ]
    )
fig.show()
fig.write_html("../IFT-6758_blogpost/_includes/plotly_Q5-1.html")

# Question 5.3

In [ ]:
import plotly.graph_objects as go

seasons = df['Season'].drop_duplicates().to_list()

def get_plot_data(dataframe, season_to_plot):
    season_df=dataframe.copy()
    season_df = season_df[season_df['Season'] == season_to_plot]
    # Create distance bins covering the range from 0 to 200
    bin_intervals = range(0, 201, 10)
    season_df['Distance_Bin'] = pd.cut(season_df['Distance'], bins=bin_intervals, right=False)
    # Group by 'ShotType' and 'Distance_Bin' and calculate the percentage of 'Goal' events within each group
    grouped_df = (
        season_df.groupby(['ShotType', 'Distance_Bin', 'Event'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )
    grouped_df['Total'] = grouped_df['Shot'] + grouped_df['Goal']
    grouped_df['Goal_Percentage'] = (grouped_df['Goal'] / grouped_df['Total']) * 100
    # Prepare data for heatmap: Goal_Percentage with Distance_Midpoint on x-axis and ShotType on y-axis
    grouped_df['Distance_Midpoint'] = grouped_df['Distance_Bin'].apply(lambda interval: interval.left + 5)  # Adjusted to represent from 0 to 190
    heatmap_data = grouped_df.pivot(index='ShotType', columns='Distance_Midpoint', values='Goal_Percentage').fillna(0) # on remplace les Nans par des 0 pour éviter les espaces vides

    return heatmap_data

# get_plot_data(df, '2016-17')
def get_plot_button(season_id, df, seasons):
    button = dict(
        args=[{ "z": [get_plot_data(df, seasons[season_id]).values],
                "text": [get_plot_data(df, seasons[season_id]).values]
               }],
        label=seasons[season_id],
        method="update"
        )
    return button

fig = go.Figure()

fig.add_trace(
    go.Heatmap(z=get_plot_data(df, seasons[0]).values,
               text=get_plot_data(df, seasons[0]).values,
               texttemplate="%{z:.1f}",
               textfont={"size":10},
               y=get_plot_data(df, seasons[0]).index,
               x=[str(i) for i in range(0, 191, 10)],
               colorscale=['#B3D4E5', '#55a3cd', '#4954b0', '#282739', '#3b2127', '#9c2f45', '#e96f36', '#E5C3B3']
    )
)

fig.update_layout(
    title={
        'text': "Goal Percentage for Shot Types and Distances",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Distance (ft)",
    yaxis_title="Shot Type",
    barmode='stack',
    width=1000,
    height=500,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)

fig.update_layout(
    updatemenus=[
        dict(
            buttons=[get_plot_button(i, df, seasons) for i in range(len(seasons))],
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
            ),
        ]
    )
fig.show()
fig.write_html("../IFT-6758_blogpost/_includes/plotly_Q5-3.html")